In [1]:
import tensorflow as tf
import numpy as np
import nltk
import json
import io
import pandas as pd
import pickle
import random

In [2]:
from google.colab import files
uploaded= files.upload()

Saving intents.json to intents.json


In [3]:
data = pd.read_json(io.StringIO(uploaded.get('intents.json').decode('utf-8')))
data

,intents
0,"{'tag': 'greeting', 'patterns': ['Hi', 'How ar..."
1,"{'tag': 'goodbye', 'patterns': ['cya', 'See yo..."
2,"{'tag': 'age', 'patterns': ['how old', 'how ol..."
3,"{'tag': 'name', 'patterns': ['what is your nam..."
4,"{'tag': 'shop', 'patterns': ['Id like to buy s..."
5,"{'tag': 'hours', 'patterns': ['when are you gu..."


In [4]:
from nltk.stem.lancaster import LancasterStemmer
stemmer= LancasterStemmer()

In [5]:
nltk.download('punkt')

try:

  with open('data.pickle','rb') as f:
    words,labels,training,output= pickle.load()

except:
  words=[]
  doc_patterns=[]
  doc_tag=[]
  labels=[]

  for intent in data['intents']:
    for pattern in intent['patterns']:
      wrd= nltk.word_tokenize(pattern)
      words.extend(wrd)
      doc_patterns.append(wrd)
      doc_tag.append(intent['tag'])
    
    if intent['tag']  not in labels:
      labels.append(intent['tag'])  

  words= [stemmer.stem(w.lower()) for w in words if w != '?']
  words= sorted(list(set(words)))
  labels= sorted(labels)

  training=[]
  output=[]
  out_empty=[0 for _ in range(len(labels))]

  for x,doc in enumerate(doc_patterns):
    bag=[]

    wrds=[stemmer.stem(w.lower()) for w in doc]

    for w in words:
      if w in wrds:
        bag.append(1)
      else:
        bag.append(0)
  
    o_row= list(out_empty)
    o_row[labels.index(doc_tag[x])]=1

    training.append(bag)
    output.append(o_row)

  training= np.array(training)
  output= np.array(output)

  with open('data.pickle','wb') as f:
    pickle.dump((words,labels,training,output),f)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(16, input_shape=(len(training[0]),), activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(len(output[0]), activation='softmax'))

In [7]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                752       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 6)                 102       
                                                                 
Total params: 1,398
Trainable params: 1,398
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(training,output,epochs=100,batch_size=8)

Epoch 1/100
4/4 [==============================] - 1s 4ms/step - loss: 1.8543 - accuracy: 0.1538
Epoch 2/100
4/4 [==============================] - 0s 3ms/step - loss: 1.8375 - accuracy: 0.1538
Epoch 3/100
4/4 [==============================] - 0s 3ms/step - loss: 1.8236 - accuracy: 0.1538
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 1.8106 - accuracy: 0.1538
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 1.7999 - accuracy: 0.1154
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 1.7891 - accuracy: 0.1923
Epoch 7/100
4/4 [==============================] - 0s 3ms/step - loss: 1.7786 - accuracy: 0.1923
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: 1.7693 - accuracy: 0.1923
Epoch 9/100
4/4 [==============================] - 0s 4ms/step - loss: 1.7599 - accuracy: 0.1538
Epoch 10/100
4/4 [==============================] - 0s 4ms/step - loss: 1.7514 - accuracy: 0.1923
Epoch 11/100
4/4 [===========

In [11]:
def bag_of_words(sent,words):
  bag=[0 for _ in range(len(words))]

  s_words= nltk.word_tokenize(sent)
  s_words= [stemmer.stem(wd.lower()) for wd in s_words]

  for s in s_words:
    for i,w in enumerate(words):
      if w==s:
        bag[i]=1
  
  return np.array(bag)

In [16]:
def chat():
  print('Chatbot is available(type quit to stop)')

  while True:
    inp= input('You: ')
    if inp.lower()== 'quit':
      break
    
    bag=bag_of_words(inp,words)
    results= model.predict(np.reshape([bag], (1, 46)))[0]
    results_index = np.argmax(results)
    
    tag= labels[results_index]

    if results[results_index] >= 0.75:

       for tg in data['intents']:
         if tg['tag']==tag:
           response= tg['responses']
   
       print(random.choice(response))

    else:
      print("I didn't that, get please try again.")
  
chat()

Chatbot is available(type quit to stop)
You: hello
Hi there, how can I help?
You: whats on the menu
We sell chocolate chip cookies for $2!
You: when are you guys open
We are open 7am-4pm Monday-Friday!
You: see you later
Goodbye!
You: quit
